In [53]:
import pandas_datareader as web
import datetime as dt

%run Util.ipynb
%run Constant.ipynb
%run AccountSavior.ipynb
%run Account.ipynb
%run Operation.ipynb

class Online_Rocket:
    year_2019_stocks = '2019_stocks_pool.txt'
    market = '000001.sz'
    
    def __init__(self,policy_name):
        self.policy_name = policy_name
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.symbols = self.get_stocks_from_pool()
        self.accountSavior = AccountSavior(is_on_prod=True)
    

    def get_near_data(self,symbol,start_date,end_date):
        data = web.get_data_yahoo(symbol,start_date,end_date)  
        return data
    
    def try_to_buy(self,cur_date,account):
        for symbol in self.symbols:
            operation = Operation(account,symbol,self.context,self.policy_name)
        
            if(operation.is_buy_point(cur_date)==True):
                operation.open_opsition(cur_date)
                
    def try_to_sell(self,cur_date,candidates,account):
        for symbol in candidates:
            operation = Operation(account,symbol,self.context,self.policy_name)
            
            if(operation.is_sell_point(cur_date)==True):
                if(self.debug==True):
                    print('try to sell '+ symbol)

                operation.sell_stock(cur_date)
        
                
    def get_to_sell_candidate_stocks(self,cur_date,account):
        symbols = list(set(account.position.keys()))
        
        return symbols
    
    
    def get_stocks_from_pool(self):
        file_path = optimizing_stocks_dir + self.year_2019_stocks
        symbols = get_all_symbols(file_path)
        
        return symbols
    
    
    def is_today_open(self,cur_date):
        data = self.get_near_data(self.market,cur_date,cur_date)
        data = data.reset_index('Date')
        data['date'] = data['Date']
        
        print(data)
        if(data[data['Date']==cur_date].shape[0]==1):
            return True
        else:
            return False
    
    
    def daily_run(self):
        print('start to daily run..')
        
        cur_date = str(dt.date.today())
        account = self.accountSavior.restore_account(cur_date)
        
        if(account is None):
            print('Warnging! account is None, init a new account!!!')
            account = Account(self.context,self.policy_name)
        
        if(self.is_today_open(cur_date)):
            account.daily_audit(cur_date)

            if(account.can_open_new_stock()==True):
                self.try_to_buy(cur_date,account)

            if(account.has_shares()==True):
                candidates = self.get_to_sell_candidate_stocks(cur_date,account)
                self.try_to_sell(cur_date,candidates,account)
        else:
            print(cur_date+' have a rest.')

        print('start to daily run done !')


In [54]:
rocket = Online_Rocket('rocket_break_v8_start_real_world_v0.1.json')  
data = rocket.daily_run()
data

start to daily run..
Warning! can not found persisted account files 2019-03-22
Warnging! account is None, init a new account!!!
        Date   High   Low   Open  Close    Volume  Adj Close       date
0 2019-03-22  12.74  12.5  12.69  12.56  44443390      12.56 2019-03-22
002411.sz Warning! No day k lines


KeyError: 'date'